In [1]:
# imports
import os
import cv2
from sklearn.ensemble import BaggingClassifier
#import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#import sklearn as skl
import sklearn.preprocessing as preproc
import numpy as np
import keras as k
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D, Flatten, Dense, Dropout, MaxPool2D
from keras.models import Sequential
#import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
#from time import time
import pandas as pd
from imblearn.over_sampling import SMOTENC
import tensorflow as tf
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
import math
import shutil
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.preprocessing import OneHotEncoder

#### variabili globali
#### GIUSEPPE
#image_path = '..\skin-cancer-mnist-ham10000\HAM10000_images_part_1'
image_path = '..\\..\\Untitled_Folder'
#### FABIO
# image_path = '..\skin-cancer-mnist-ham10000\HAM10000_images_part_1'
csv_path = 'HAM10000_metadata.csv'
csv_completo = 'dataframe_completo.csv'
csv_completo_2 = 'dataframe_completo_2.csv' 

Using TensorFlow backend.


In [2]:
dataset = pd.read_csv(csv_completo_2, encoding = "ISO-8859-1")
batch = 1000

def my_train_batch(dataset, batch, epochs, classifier):
    size = len(dataset)
    epoch=0
    cl1 = k.models.load_model(classifier)
    # train del classificatore un batch alla volta
    while size >= batch:
        # To get n random rows 
        samples = dataset.sample(n = batch)
        # splitting test and validation
        X_train, X_val, y_train, y_val = train_test_split(samples["image_id"], samples["dx"], test_size=0.30)
        
        train_img = np.asarray(load_images(X_train))
        val_img = np.asarray(load_images(X_val))
        
        #1hot encoder
        enc = OneHotEncoder(sparse=False)
        y_train_shaped = np.array(y_train).reshape(-1,1)
        y_val_shaped = np.array(y_val).reshape(-1,1)
        enc.fit(y_train_shaped)
        y_train_one = enc.transform(y_train_shaped)
        y_val_one = enc.transform(y_val_shaped)
        
        cl1.fit(train_img, y_train_one, epochs=epochs, shuffle=True, validation_data=(val_img, y_val_one), verbose=1)
        
        #pulizia memoria e reset per il prossimo ciclo
        del X_train, y_train_one, X_val, y_val_one, train_img, val_img
        dataset = dataset.drop(samples.index)
        size = len(dataset)
        del samples, enc
        epoch = epoch + 1
        print("fine epoca " + str(epoch) + ";\trestano " + str(size) + " campioni nel dataset")
    cl1.save(classifier)
    return

# dal dataframe ne estrae le immagini in un array, serve nel batch
def load_images(array):
    images = list()
    #print(array.iloc[0])
    for index in range(0,len(array)):
        #print(index.type())
        img_id = array.iloc[index]
        elem = os.path.join(image_path, img_id)
        elem = elem + ".jpg"
        img = cv2.imread(elem)
        # TODO va fatto un reshape, non capisco perché
        images.append(np.asarray(img)) #########################################################################################
    return images

def new_classifier3():
    # Set the CNN model 
    input_shape = (150, 200, 3)
    num_classes = 7
    
    model = Sequential()
    model.add(Conv2D(20, kernel_size=(5, 5),activation='relu',padding = 'Same', input_shape = input_shape))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(140,kernel_size=(3, 3), activation='relu',padding = 'Same'))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(50,kernel_size=(3, 3), activation='relu',padding = 'Same'))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()
    model.compile(optimizer='adadelta', loss='binary_crossentropy')
    return model

def kaggle_classifier():
    # Set the CNN model 
    # my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out
    input_shape = (150, 200, 3)
    num_classes = 7
    
    model = Sequential()
    model.add(Conv2D(16, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=input_shape))
    model.add(Conv2D(16,kernel_size=(3, 3), activation='relu',padding = 'Same',))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(32, (3, 3), activation='relu',padding = 'Same'))
    model.add(Conv2D(32, (3, 3), activation='relu',padding = 'Same'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.40))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.summary()
    model.compile(optimizer='adadelta', loss='binary_crossentropy')
    return model

In [15]:
### inizializzazione classificatore sul disco
cl1 = new_classifier3()
cl1.save('cl1.h5')

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 150, 200, 20)      1520      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 75, 100, 20)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 75, 100, 20)       80        
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 75, 100, 140)      25340     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 37, 50, 140)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 37, 50, 140)       560       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 37, 50, 50)       

In [ ]:
mass = int(len(dataset)*0.7)
samples = dataset.sample(n=mass)
my_train_batch(dataset=samples, batch=10000, epochs=20, classifier = 'classifier1.h5')

In [ ]:
##### controllo dimensioni immagini nella cartella
legit = (200, 150)
i = 0
for elem in dataset["image_id"]:
    fname = elem + ".jpg"
    opened = Image.open(os.path.join(image_path, fname))
    if opened.size != legit:
        print(opened.size)
    opened.close()
    i = i+1
    if i % 5000 == 0:
        print(str(i) + " iterate")

In [3]:
### inizializzazione classificatore sul disco
cl_name = 'training_batch_cl1.h5'

cl1 = new_classifier3()
cl1.save(cl_name)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 200, 20)      1520      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 100, 20)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 75, 100, 20)       80        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 100, 140)      25340     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 37, 50, 140)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 37, 50, 140)       560       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 37, 50, 50)       

In [5]:
mass = int(len(dataset)*0.7)
samples = dataset.sample(n=mass)
my_train_batch(dataset=samples, batch=6000, epochs=5, classifier = cl_name)

Train on 4200 samples, validate on 1800 samples
Epoch 1/5
4200/4200 [==============================] - 172s 41ms/step - loss: 0.5001 - val_loss: 0.4817
Epoch 2/5
4200/4200 [==============================] - 179s 43ms/step - loss: 0.3069 - val_loss: 0.3369
Epoch 3/5
4200/4200 [==============================] - 179s 43ms/step - loss: 0.2849 - val_loss: 0.4828
Epoch 4/5
4200/4200 [==============================] - 178s 42ms/step - loss: 0.2627 - val_loss: 1.4253
Epoch 5/5
4200/4200 [==============================] - 179s 43ms/step - loss: 0.2393 - val_loss: 0.6021
fine epoca 1;	restano 27568 campioni nel dataset
Train on 4200 samples, validate on 1800 samples
Epoch 1/5
4200/4200 [==============================] - 179s 43ms/step - loss: 0.2676 - val_loss: 0.3051
Epoch 2/5
4200/4200 [==============================] - 3056s 728ms/step - loss: 0.2398 - val_loss: 3.5680
Epoch 3/5
4200/4200 [==============================] - 155s 37ms/step - loss: 0.2238 - val_loss: 1.3627
Epoch 4/5
4200/4200 [

In [6]:
cl1.save(cl_name)

In [3]:
images = np.asarray(load_images(dataset["image_id"]))

In [9]:
X_train1, X_test, y_train1, y_test = train_test_split(images, dataset["dx"], test_size=0.30)
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train1, test_size=0.30)

#1hot encoder
enc = OneHotEncoder(sparse=False)
y_train_shaped = np.array(y_train).reshape(-1,1)
y_val_shaped = np.array(y_val).reshape(-1,1)
enc.fit(y_train_shaped)
y_train_one = enc.transform(y_train_shaped)
y_val_one = enc.transform(y_val_shaped)

NameError: name 'epochs' is not defined

In [16]:
cl1.fit(X_train, y_train_one, epochs=5, shuffle=True, validation_data=(X_val, y_val_one), verbose=1)

Train on 23497 samples, validate on 10071 samples
Epoch 1/5
23497/23497 [==============================] - 1092s 46ms/step - loss: 0.3254 - val_loss: 0.3121
Epoch 2/5
23497/23497 [==============================] - 1058s 45ms/step - loss: 0.2507 - val_loss: 0.3938
Epoch 3/5
23497/23497 [==============================] - 1020s 43ms/step - loss: 0.2261 - val_loss: 0.3642
Epoch 4/5
23497/23497 [==============================] - 1023s 44ms/step - loss: 0.2036 - val_loss: 0.3431
Epoch 5/5
23497/23497 [==============================] - 1026s 44ms/step - loss: 0.1867 - val_loss: 0.3544


In [17]:
cl1.save("TRAINED_nc3.h5")